## Part1

$X$ can be any $n \times m$ matrix. Then $X^\top X$ is $m \times m$ matrix.

If $Xv = 0$, then $X^\top Xv = X^\top (Xv) = X^\top (0) = 0$. So we can get $\ker(X) \subseteq \ker(X^\top X)$.

If $X^\top Xc = 0$, we can get $c^\top X^\top Xc = 0$. Next we can have
$$
c^\top X^\top Xc \; = c^\top (X^\top X)c \;=\; (Xc)^\top (Xc) \;=\; \|Xc\|^2 \;=\; 0
$$
so $\ker(X^\top X) \subseteq \ker(X)$

As a result, $\ker(X) = \ker(X^\top X)$.

$$
\text{rank}(X) 
= m - \dim \ker(X) 
= m - \dim \ker(X^\top X) 
= \text{rank}(X^\top X)
$$


$$
\boxed{\text{rank}(X) = \text{rank}(X^\top X)}
$$

## Part2

In function pinv, I used full SVD calculation. Using thin SVD will get the same result. The cutoff value can be changed based on cases. Here I used 0.00000001, which is suitable for most use cases.

In [1]:
import numpy as np

def pinv(X):
    #Do the SVD to the X first, transpose U and Vt to get U_t and V, which is directly needed to calculate pseudoinverse. X_T is needed for its shape.
    U, S, Vt = np.linalg.svd(X, full_matrices=True)
    X_T = np.transpose(X)
    V = np.transpose(Vt)
    U_T = np.transpose(U)
    
    #Setting the cutoff at 1e-8 means all singular values smaller than 0.00000001 are treated as 0 and put the zero in inv_S to avoid Numerical explosion 
    #caused by zero or too small singular value, which makes our function work when X is not full rank.
    cutoff = 1e-8
    inv_S = np.zeros_like(S)
    #In this way we get the number in Σ⁺. Zero and closed to zero values are zero at their position.
    inv_S[S > cutoff] = 1.0 / S[S > cutoff]
    
    # Σ⁺ should be (n, m), which is same shape of X_T, so we generate all zero (n,m) matrix at first.
    Sigma = np.zeros_like(X_T, dtype=float)
    
    #Fill the diagonal with inv_S to get Σ⁺
    np.fill_diagonal(Sigma, inv_S)
    
    #Calculte the Moore-Penrose pseudoinverse
    pseu = V @ Sigma @ U_T
    return pseu

def lr(X, y):
    #coe is coefficient, and res is  residual
    coe = pinv(X) @ y
    res = y - X @coe
    return coe, res

In [2]:
import random

Test the functions.

In [3]:
n = 1000
p = 3
def rnorm(n, mean = 0, sd = 1):
    return mean + sd * np.random.normal(0, 1, n)

x1 = rnorm(n)
x2 = rnorm(n)
x3 = rnorm(n)
y = 5 * x1 - 17 * x2 + 8 * x3 + 10 * rnorm(n)
X = np.column_stack((x1, x2, x3))

In [4]:
X

array([[ 0.44918735, -0.41606898, -0.10411676],
       [-0.90596016, -0.52050273,  1.27735033],
       [ 1.28957115, -0.39580329, -0.19536227],
       ...,
       [ 0.90826921, -0.58853757, -2.23592566],
       [-0.21061236, -2.28368355, -0.29550583],
       [ 1.68025911,  1.23053594, -0.22417219]])

In [5]:
p = pinv(X)

In [6]:
p

array([[ 0.00042815, -0.00093074,  0.00123283, ...,  0.00096784,
        -0.00021094,  0.00162391],
       [-0.00039261, -0.00055267, -0.00036095, ..., -0.00048489,
        -0.00219306,  0.00121121],
       [-0.00010243,  0.00121252, -0.00022654, ..., -0.00204076,
        -0.00018223, -0.00032421]])

In [7]:
coe, r = lr(X, y)

In [8]:
coe

array([  4.87068604, -17.38098016,   7.86841838])

In [9]:
r

array([ 2.01717533e+00,  4.72712397e+00, -8.03963578e-01,  6.32887070e+00,
       -2.04084489e+01, -4.09289054e+00, -1.60327980e+01,  1.74260811e+00,
       -1.71775339e+00, -6.52155918e+00,  8.55824915e+00, -4.47147251e+00,
       -4.52432334e+00,  1.97424780e+01, -5.95324861e-01,  8.32284945e+00,
       -8.46474725e+00,  2.69898508e+00, -1.11665368e+00,  1.23028326e+01,
        8.54167454e+00, -2.17532624e+01,  1.13958971e+01,  1.02651114e+01,
        5.72845350e+00,  1.45427379e+01,  8.12166186e+00,  9.57466148e+00,
       -6.57982611e+00, -6.98531405e-02, -9.07918862e+00,  7.69413118e+00,
       -1.57552716e+01,  1.63226995e+01, -1.39153470e+01,  1.80187040e+01,
        1.07091390e+00, -7.97690468e-01,  5.79507445e+00, -1.38099014e+00,
        5.71042811e+00, -7.61986513e+00, -1.21866189e+01,  1.46160908e+01,
        7.49688053e+00,  4.77539043e+00,  1.84201423e+01, -2.54385324e+00,
       -6.47002942e+00,  9.18803583e+00,  7.35206564e+00,  8.97793706e-01,
       -4.84978001e-01, -